- 패키지 불러오기

In [2]:
import torch

In [3]:
# -*-coding: utf-8-*-

import pandas as pd # 데이터 프레임 패키지
import numpy as np  # 수학적 연산을 위한 패키지
from sklearn.model_selection import train_test_split  # train, test로 데이터 split 함수

import lightgbm as lgb # lightgbm 패키지
from lightgbm import plot_importance # lightgbm의 변수중요도 패키지
import xgboost as xgb # xgboost 패키지
from sklearn.ensemble import RandomForestClassifier # random forest 패키지


import seaborn as sns # 그래프를 그려주는 패키지
import matplotlib.pyplot as plt  # 그래프를 그려주는 패키지
import matplotlib.font_manager as fm  # 그래프 폰트 지정 패키지
get_ipython().run_line_magic('matplotlib', 'inline')

import time, datetime  # 시간 관련 패키지 1
from pytz import timezone, utc # 시간 관련 패키지 2

from sklearn.model_selection import cross_val_score  # CV 관련 패키지
from sklearn.model_selection import GridSearchCV # CV grid search관련 패키지

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve, make_scorer
from sklearn.preprocessing import LabelEncoder  # str 형식의 데이터 encoding 함수
import sklearn.externals
import joblib # pickle 파일 저장 패키지

import pyodbc  # hive 연결 패키지
import gc, sys, os # python, 시스템 관련 패키지
gc.enable() # 파이썬 관리 기능
import warnings # 경고 관련 패키지
warnings.filterwarnings('ignore') # 경고 무시 기능


sys.path.insert(1, '/home/cdsw/FS/common/') # 사용자 정의 함수 경로 
from FS            import FS_filter, FS_wrapper     # 변수 선택 관련 함수
from comm_Function import commFunc                  # 변수 처리 관련 함수
from ml_lib_nocatbo        import multi_col_label_encoder   # 라벨 인코딩 함수
from ml_lib_nocatbo import MlAlgoRun, get_clf_eval_metrics, feat_imp, roc_plot, lift_result, experiment_report


Bad key "backend.qt4" on line 43 in
/etc/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


### 경로설정

In [4]:
# path 
filedir= "/home/cdsw/100.RL"  # 기본 경로
log_path = os.path.join(filedir,'logs') # 로그 경로
dat_path = os.path.join(filedir,'Data') # 데이터 저장 경로
mdl_path = os.path.join(filedir,'Model') # 모델 저장 경로
res_path = os.path.join(filedir,'Result') # 모델 결과 저장 경로

# create directories
try:
    if not(os.path.isdir(dat_path)): # 데이터 저장경로가 없을 경우 해당 경로에 폴더 생성
        os.makedirs(dat_path)
    if not(os.path.isdir(mdl_path)): # 모델 저장경로가 없을 경우 해당 경로에 폴더 생성
        os.makedirs(mdl_path)
    if not(os.path.isdir(res_path)): # 모델 결과 저장경로가 없을 경우 해당 경로에 폴더 생성
        os.makedirs(res_path)
    if not(os.path.isdir(log_path)): # 로그 경로가 없을 경우 해당 경로에 폴더 생성
        os.makedirs(log_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directories!")
        raise


### 마트 불러오기 

### HIVE 접속

In [4]:
# Hive 접속 환경 설정 
os.environ['ODBCINI'] = '/opt/cloudera/hiveodbc/Setup/odbc.ini'
os.environ['ODBCSYSINI'] = '/opt/cloudera/hiveodbc/Setup/'
os.environ['CLOUDERAHIVEODBCINI'] = \
    '/opt/cloudera/hiveodbc/lib/64/cloudera.hiveodbc.ini'

# Hive connection 설정
hive_conn = pyodbc.connect(
    "DSN=HiveDSN",
    driver = 'Cloudera ODBC Driver for Apache Hive 64-bit',
    autocommit = True
)

### 마트 로딩

In [6]:
# 마트 pt1 저장 경로
mart_pt1_nm = 'ROLL_MART_1' 
mart_pt1_path = os.path.join(dat_path, mart_pt1_nm+'.pkl')

In [7]:
mart_pt1 = pd.read_pickle(os.path.join(dat_path, mart_pt1_nm + '.pkl'))

In [8]:
print(mart_pt1.shape)
mart_pt1.head()

(3068027, 21)


,wk,wk_a1,kb페이id,리워드금액,리워드구간대,f_출석체크,m_출석체크,w0_kbp건수,w0_kbp금액,w1_kbp건수,...,w0_kbp로그인횟수,w1_kbp로그인여부,w1_kbp로그인횟수,w0_kbp건수_ntile,w0_kbp금액_ntile,w0_kbp로그인횟수_ntile,w1_kbp건수_ntile,w1_kbp금액_ntile,w1_kbp로그인횟수_ntile,연령그룹
0,25,26,K001681120,63.0,2,1,63.0,0,0,0,...,0,0,0,1,1,1,1,1,1,50대이상
1,25,26,K007207873,66.0,2,1,66.0,3,31520,0,...,54,0,0,4,3,4,1,1,1,3040
2,25,26,K006509489,68.0,2,3,120.0,5,53650,0,...,52,0,0,4,4,4,1,1,1,3040
3,25,26,K004342331,98.0,4,1,98.0,1,1900,0,...,51,0,0,3,3,4,1,1,1,20대이하
4,25,26,K007227403,12.0,1,1,12.0,0,0,0,...,51,0,0,1,1,4,1,1,1,20대이하


In [7]:
mart_pt1.to_csv('/home/cdsw/rl_mart_pt1.csv',index=False,header=True)

In [6]:
# 마트 pt1 저장 경로
mart_pt1_nm = 'ROLL_MART_1' 
mart_pt1_path = os.path.join(dat_path, mart_pt1_nm+'.pkl')

# 마트 로딩(마트 저장 경로에 마트가 없을 경우 HIVE에서, 마트가 있을경우는 CDSW에서 로딩)
sql = f"select * from PK119549.ROLL_STEP04 "
mart_pt1 = pd.read_sql(sql, hive_conn)
mart_pt1.columns = [cols.split(".")[1] for cols in mart_pt1.columns]
mart_pt1.to_pickle(os.path.join(dat_path, mart_pt1_nm + '.pkl'))

In [9]:
print(mart_pt1.columns.to_list())

['wk', 'wk_a1', 'kb페이id', '리워드금액', '리워드구간대', 'f_출석체크', 'm_출석체크', 'w0_kbp건수', 'w0_kbp금액', 'w1_kbp건수', 'w1_kbp금액', 'w0_kbp로그인횟수', 'w1_kbp로그인여부', 'w1_kbp로그인횟수', 'w0_kbp건수_ntile', 'w0_kbp금액_ntile', 'w0_kbp로그인횟수_ntile', 'w1_kbp건수_ntile', 'w1_kbp금액_ntile', 'w1_kbp로그인횟수_ntile', '연령그룹']


# State Space 탐색

In [10]:
mart_pt1['w0_kbp로그인여부'] ='1' 

In [11]:
mart_pt1.groupby(['리워드구간대', 'w0_kbp건수_ntile', 'w0_kbp금액_ntile', 'w0_kbp로그인횟수_ntile','w0_kbp로그인여부', 'w1_kbp건수_ntile', 'w1_kbp금액_ntile', 'w1_kbp로그인횟수_ntile','w1_kbp로그인여부','연령그룹', ]).agg({'kb페이id':'count'}).unstack('리워드구간대')

kb페이id  \
리워드구간대                                                                                                                              1   
w0_kbp건수_ntile w0_kbp금액_ntile w0_kbp로그인횟수_ntile w0_kbp로그인여부 w1_kbp건수_ntile w1_kbp금액_ntile w1_kbp로그인횟수_ntile w1_kbp로그인여부 연령그룹            
1              1              1                 1           1              1              1                 0           20대이하  2106.0   
                                                                                                                        3040   5126.0   
                                                                                                                        50대이상  2611.0   
                                                                                                            1           20대이하   524.0   
                                                                                                                        3040   1198.0   
...                                                                                                                               ...   
4              4              4                 1           4              4              3                 1           3040   2919.0   
                                                                                                                        50대이상   598.0   
                                                                                          4                 1           20대이하  2480.0   
                                                                                                                        3040   8592.0   
                                                                                                                        50대이상  1646.0   

                                                                                                                                        \
리워드구간대                                                                                                                               2   
w0_kbp건수_ntile w0_kbp금액_ntile w0_kbp로그인횟수_ntile w0_kbp로그인여부 w1_kbp건수_ntile w1_kbp금액_ntile w1_kbp로그인횟수_ntile w1_kbp로그인여부 연령그룹             
1              1              1                 1           1              1              1                 0           20대이하   1885.0   
                                                                                                                        3040    4657.0   
                                                                                                                        50대이상   2422.0   
                                                                                                            1           20대이하    498.0   
                                                                                                                        3040    1127.0   
...                                                                                                                                ...   
4              4              4                 1           4              4              3                 1           3040    5783.0   
                                                                                                                        50대이상    996.0   
                                                                                          4                 1           20대이하   5151.0   
                                                                                                                        3040   30299.0   
                                                                                                                        50대이상   6265.0   

                                                                                                                                        \
리워드구간대                                                                                                                               3   

In [14]:
mart_pt1['연령그룹'].value_counts()

3040     1870400
50대이상     628276
20대이하     569351
Name: 연령그룹, dtype: int64

- state : 로그인횟수(4) * 금액대 (4) * 로그인여부(2) * 연령(3)

In [16]:
mart_pt1.groupby(['리워드구간대', 'w0_kbp로그인횟수_ntile', 'w0_kbp금액_ntile', 'w0_kbp로그인여부', 'w1_kbp로그인횟수_ntile', 'w1_kbp금액_ntile', 'w1_kbp로그인여부','연령그룹']).agg({'kb페이id':'count'}).unstack('리워드구간대')

kb페이id  \
리워드구간대                                                                                                 1   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하   6513.0   
                                                                                          3040   16013.0   
                                                                                          50대이상   8359.0   
                                                                              1           20대이하   1307.0   
                                                                                          3040    2623.0   
...                                                                                                  ...   
4                 4              1           4                 3              1           3040     948.0   
                                                                                          50대이상    204.0   
                                                               4              1           20대이하   2628.0   
                                                                                          3040    8978.0   
                                                                                          50대이상   1780.0   

                                                                                                          \
리워드구간대                                                                                                 2   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하   5630.0   
                                                                                          3040   14430.0   
                                                                                          50대이상   7400.0   
                                                                              1           20대이하   1349.0   
                                                                                          3040    2556.0   
...                                                                                                  ...   
4                 4              1           4                 3              1           3040    5806.0   
                                                                                          50대이상   1400.0   
                                                               4              1           20대이하   5625.0   
                                                                                          3040   32741.0   
                                                                                          50대이상   6989.0   

                                                                                                          \
리워드구간대                                                                                                 3   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하   2575.0   
                                                                                          3040    5910.0   
                                                                                          50대이상   3185.0   
                                                                              1           20대이하    682.0   
                                                                                          3040    1301.0   
...                                                                                                  ...   
4                 4              1           4                 3              1           3040   108

In [15]:
print(4*4*2*4*4*2*3)

3072


- 연령대를 빼면?

In [17]:
mart_pt1.groupby(['리워드구간대', 'w0_kbp로그인횟수_ntile', 'w0_kbp금액_ntile', 'w0_kbp로그인여부', 'w1_kbp로그인횟수_ntile', 'w1_kbp금액_ntile', 'w1_kbp로그인여부']).agg({'kb페이id':'count'}).unstack('리워드구간대')

kb페이id  \
리워드구간대                                                                                           1   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부            
1                 1              1           1                 1              0            30885.0   
                                                                              1             5129.0   
                                                               2              0            21381.0   
                                                                              1             3256.0   
                                                               3              0            10552.0   
...                                                                                            ...   
4                 4              1           3                 4              1             4902.0   
                                             4                 1              1              313.0   
                                                               2              1              157.0   
                                                               3              1             1584.0   
                                                               4              1            13386.0   

                                                                                                    \
리워드구간대                                                                                           2   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부            
1                 1              1           1                 1              0            27460.0   
                                                                              1             5036.0   
                                                               2              0            18685.0   
                                                                              1             3700.0   
                                                               3              0             9238.0   
...                                                                                            ...   
4                 4              1           3                 4              1            10091.0   
                                             4                 1              1             1975.0   
                                                               2              1             1002.0   
                                                               3              1             8391.0   
                                                               4              1            45355.0   

                                                                                                    \
리워드구간대                                                                                           3   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부            
1                 1              1           1                 1              0            11670.0   
                                                                              1             2554.0   
                                                               2              0             8075.0   
                                                                              1             2082.0   
                                                               3              0             4056.0   
...                                                                                            ...   
4                 4              1           3                 4              1            11236.0   
                                             4                 1              1             3452.0   
                                                               2              1             1758.0   
          

In [18]:
print(1117/3)

372.3333333333333


- 일단 전체를 DataFrame으로 만들고 어디가 빠졌는지 보기

In [41]:
mart_gr1 = mart_pt1.groupby(['리워드구간대', 'w0_kbp로그인횟수_ntile', 'w0_kbp금액_ntile', 'w0_kbp로그인여부', 'w1_kbp로그인횟수_ntile', 'w1_kbp금액_ntile', 'w1_kbp로그인여부','연령그룹'])\
.agg({'리워드구간대':'count'})\
.unstack('리워드구간대')\
.fillna(0)


In [26]:
mart_gr1.head()

리워드구간대  \
리워드구간대                                                                                                 1   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하   6513.0   
                                                                                          3040   16013.0   
                                                                                          50대이상   8359.0   
                                                                              1           20대이하   1307.0   
                                                                                          3040    2623.0   

                                                                                                          \
리워드구간대                                                                                                 2   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하   5630.0   
                                                                                          3040   14430.0   
                                                                                          50대이상   7400.0   
                                                                              1           20대이하   1349.0   
                                                                                          3040    2556.0   

                                                                                                         \
리워드구간대                                                                                                3   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹            
1                 1              1           1                 1              0           20대이하  2575.0   
                                                                                          3040   5910.0   
                                                                                          50대이상  3185.0   
                                                                              1           20대이하   682.0   
                                                                                          3040   1301.0   

                                                                                                         
리워드구간대                                                                                                4  
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹           
1                 1              1           1                 1              0           20대이하  1676.0  
                                                                                          3040   4018.0  
                                                                                          50대이상  2101.0  
                                                                              1           20대이하   432.0  
                                                                                          3040    951.0

In [27]:
mart_gr1.columns

MultiIndex([('리워드구간대', 1),
            ('리워드구간대', 2),
            ('리워드구간대', 3),
            ('리워드구간대', 4)],
           names=[None, '리워드구간대'])

In [33]:
mart_gr1.columns.to_list()

[('리워드구간대', 1), ('리워드구간대', 2), ('리워드구간대', 3), ('리워드구간대', 4)]

In [34]:
print(mart_gr1.columns.values)

[('리워드구간대', 1) ('리워드구간대', 2) ('리워드구간대', 3) ('리워드구간대', 4)]


In [42]:
mart_gr1.columns.values

array([('리워드구간대', 1), ('리워드구간대', 2), ('리워드구간대', 3), ('리워드구간대', 4)],
      dtype=object)

In [54]:
mart_gr1.head()

리워드구간대  \
리워드구간대                                                                                                 1   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하   6513.0   
                                                                                          3040   16013.0   
                                                                                          50대이상   8359.0   
                                                                              1           20대이하   1307.0   
                                                                                          3040    2623.0   

                                                                                                          \
리워드구간대                                                                                                 2   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하   5630.0   
                                                                                          3040   14430.0   
                                                                                          50대이상   7400.0   
                                                                              1           20대이하   1349.0   
                                                                                          3040    2556.0   

                                                                                                         \
리워드구간대                                                                                                3   
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹            
1                 1              1           1                 1              0           20대이하  2575.0   
                                                                                          3040   5910.0   
                                                                                          50대이상  3185.0   
                                                                              1           20대이하   682.0   
                                                                                          3040   1301.0   

                                                                                                         
리워드구간대                                                                                                4  
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹           
1                 1              1           1                 1              0           20대이하  1676.0  
                                                                                          3040   4018.0  
                                                                                          50대이상  2101.0  
                                                                              1           20대이하   432.0  
                                                                                          3040    951.0

- 컬럼명변경 : 리워드구간대가 str이 아녀서 오류남

In [44]:
print(['_'.join(col) for col in mart_gr1.columns.to_list()])

TypeError: sequence item 1: expected str instance, int found

In [52]:
tst = [('tst',1)]

In [48]:
tst2= [('tst','1')]

In [49]:
print(['_'.join(col) for col in tst2])

['tst_1']


In [53]:
print(['_'.join(col) for col in tst])

TypeError: sequence item 1: expected str instance, int found

- 컬럼명 변경 : 리워드 구간대만 str으로 바꿔주기 

In [55]:
col_tst = mart_gr1.columns.to_list()
print(col_tst)

[('리워드구간대', 1), ('리워드구간대', 2), ('리워드구간대', 3), ('리워드구간대', 4)]


In [58]:
for i in col_tst:
    print(i)

('리워드구간대', 1)
('리워드구간대', 2)
('리워드구간대', 3)
('리워드구간대', 4)


In [64]:
for i in col_tst:
    print(i[0]+'_'+str(i[1]))

리워드구간대_1
리워드구간대_2
리워드구간대_3
리워드구간대_4


In [66]:
print([i[0]+'_'+str(i[1]) for i in col_tst])

['리워드구간대_1', '리워드구간대_2', '리워드구간대_3', '리워드구간대_4']


In [68]:
mart_gr1.columns = [i[0]+'_'+str(i[1]) for i in mart_gr1.columns.to_list()]

In [69]:
mart_gr1.head()

리워드구간대_1  \
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹              
1                 1              1           1                 1              0           20대이하    6513.0   
                                                                                          3040    16013.0   
                                                                                          50대이상    8359.0   
                                                                              1           20대이하    1307.0   
                                                                                          3040     2623.0   

                                                                                                 리워드구간대_2  \
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹              
1                 1              1           1                 1              0           20대이하    5630.0   
                                                                                          3040    14430.0   
                                                                                          50대이상    7400.0   
                                                                              1           20대이하    1349.0   
                                                                                          3040     2556.0   

                                                                                                 리워드구간대_3  \
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹              
1                 1              1           1                 1              0           20대이하    2575.0   
                                                                                          3040     5910.0   
                                                                                          50대이상    3185.0   
                                                                              1           20대이하     682.0   
                                                                                          3040     1301.0   

                                                                                                 리워드구간대_4  
w0_kbp로그인횟수_ntile w0_kbp금액_ntile w0_kbp로그인여부 w1_kbp로그인횟수_ntile w1_kbp금액_ntile w1_kbp로그인여부 연령그룹             
1                 1              1           1                 1              0           20대이하    1676.0  
                                                                                          3040     4018.0  
                                                                                          50대이상    2101.0  
                                                                              1           20대이하     432.0  
                                                                                          3040      951.0

In [70]:
mart_gr1_copy1 = mart_gr1.copy()

In [71]:
mart_gr1_copy1.reset_index(inplace=True)
mart_gr1_copy1.head()

,w0_kbp로그인횟수_ntile,w0_kbp금액_ntile,w0_kbp로그인여부,w1_kbp로그인횟수_ntile,w1_kbp금액_ntile,w1_kbp로그인여부,연령그룹,리워드구간대_1,리워드구간대_2,리워드구간대_3,리워드구간대_4
0,1,1,1,1,1,0,20대이하,6513.0,5630.0,2575.0,1676.0
1,1,1,1,1,1,0,3040,16013.0,14430.0,5910.0,4018.0
2,1,1,1,1,1,0,50대이상,8359.0,7400.0,3185.0,2101.0
3,1,1,1,1,1,1,20대이하,1307.0,1349.0,682.0,432.0
4,1,1,1,1,1,1,3040,2623.0,2556.0,1301.0,951.0


In [72]:
print(mart_gr1_copy1.shape)

(1117, 11)


In [73]:
mart_gr1_copy1.dtypes

w0_kbp로그인횟수_ntile      int64
w0_kbp금액_ntile         int64
w0_kbp로그인여부           object
w1_kbp로그인횟수_ntile      int64
w1_kbp금액_ntile         int64
w1_kbp로그인여부           object
연령그룹                  object
리워드구간대_1             float64
리워드구간대_2             float64
리워드구간대_3             float64
리워드구간대_4             float64
dtype: object

In [74]:
mart_gr1_copy1['tst'] = mart_gr1_copy1['w0_kbp로그인횟수_ntile'] + mart_gr1_copy1['w0_kbp금액_ntile']
mart_gr1_copy1.head()

,w0_kbp로그인횟수_ntile,w0_kbp금액_ntile,w0_kbp로그인여부,w1_kbp로그인횟수_ntile,w1_kbp금액_ntile,w1_kbp로그인여부,연령그룹,리워드구간대_1,리워드구간대_2,리워드구간대_3,리워드구간대_4,tst
0,1,1,1,1,1,0,20대이하,6513.0,5630.0,2575.0,1676.0,2
1,1,1,1,1,1,0,3040,16013.0,14430.0,5910.0,4018.0,2
2,1,1,1,1,1,0,50대이상,8359.0,7400.0,3185.0,2101.0,2
3,1,1,1,1,1,1,20대이하,1307.0,1349.0,682.0,432.0,2
4,1,1,1,1,1,1,3040,2623.0,2556.0,1301.0,951.0,2


In [ ]:
['리워드금액', '리워드구간대', 'f_출석체크', 'm_출석체크', 'w0_kbp건수', 'w0_kbp금액', 'w1_kbp건수', 'w1_kbp금액', 'w0_kbp로그인횟수', 'w1_kbp로그인여부', 'w1_kbp로그인횟수', 'w0_kbp건수_ntile', 'w0_kbp금액_ntile', 'w0_kbp로그인횟수_ntile', 'w1_kbp건수_ntile', 'w1_kbp금액_ntile', 'w1_kbp로그인횟수_ntile', '연령그룹']

In [ ]:
'w0_kbp건수_ntile', 'w0_kbp금액_ntile', 'w0_kbp로그인횟수_ntile', 'w1_kbp건수_ntile', 'w1_kbp금액_ntile', 'w1_kbp로그인횟수_ntile', '연령그룹'

In [75]:
col_state0= ['w0_kbp로그인여부','w0_kbp로그인횟수_ntile','w0_kbp금액_ntile','연령그룹']
col_state1= ['w1_kbp로그인여부','w1_kbp로그인횟수_ntile','w1_kbp금액_ntile','연령그룹']
col_state0_convert= ['w0_kbp로그인여부','w0_kbp로그인횟수_ntile','w0_kbp금액_ntile']
col_state1_convert= ['w1_kbp로그인여부','w1_kbp로그인횟수_ntile','w1_kbp금액_ntile']

In [76]:
# State_T0 타입변환 
State_T0_convert = pd.DataFrame(mart_gr1_copy1[col_state0_convert].values.astype(str),columns=col_state0_convert)
# State_T1 타입변환 
State_T1_convert = pd.DataFrame(mart_gr1_copy1[col_state1_convert].values.astype(str),columns=col_state1_convert)

In [77]:
State_T0_convert.head()

,w0_kbp로그인여부,w0_kbp로그인횟수_ntile,w0_kbp금액_ntile
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1


In [78]:
State_T0_convert['s0']= State_T0_convert['w0_kbp로그인여부']+State_T0_convert['w0_kbp로그인횟수_ntile']+State_T0_convert['w0_kbp금액_ntile']

In [79]:
State_T0_convert.head()

,w0_kbp로그인여부,w0_kbp로그인횟수_ntile,w0_kbp금액_ntile,s0
0,1,1,1,111
1,1,1,1,111
2,1,1,1,111
3,1,1,1,111
4,1,1,1,111


In [80]:
State_T0_convert.tail()

,w0_kbp로그인여부,w0_kbp로그인횟수_ntile,w0_kbp금액_ntile,s0
1112,1,4,4,144
1113,1,4,4,144
1114,1,4,4,144
1115,1,4,4,144
1116,1,4,4,144


In [92]:
multi_col_label_encoder

<function ml_lib_nocatbo.multi_col_label_encoder(X, cols=None)>

- 연령그룹 인코딩

In [94]:
# 문자형 변수 라벨 인코딩
mart_gr1_copy1, label_dict = multi_col_label_encoder(mart_gr1_copy1, ['연령그룹'])

# 라벨 인코딩 결과 딕셔너리 저장
np.save(os.path.join(dat_path, 'age_label_dict.npy'), label_dict)

연령그룹  - Completed


In [95]:
label_dict

{'연령그룹': {0: 0, 1: 1, 2: 2}}

In [96]:
mart_gr1_copy1.head()

,w0_kbp로그인횟수_ntile,w0_kbp금액_ntile,w0_kbp로그인여부,w1_kbp로그인횟수_ntile,w1_kbp금액_ntile,w1_kbp로그인여부,연령그룹,리워드구간대_1,리워드구간대_2,리워드구간대_3,리워드구간대_4,tst
0,1,1,1,1,1,0,0,6513.0,5630.0,2575.0,1676.0,2
1,1,1,1,1,1,0,1,16013.0,14430.0,5910.0,4018.0,2
2,1,1,1,1,1,0,2,8359.0,7400.0,3185.0,2101.0,2
3,1,1,1,1,1,1,0,1307.0,1349.0,682.0,432.0,2
4,1,1,1,1,1,1,1,2623.0,2556.0,1301.0,951.0,2


- state 0에서 빠진건 없음

In [82]:
State_T0_convert['s0'].unique()

array(['111', '112', '113', '114', '121', '122', '123', '124', '131',
       '132', '133', '134', '141', '142', '143', '144'], dtype=object)

In [86]:
print(len(State_T0_convert['s0'].unique()))

16


- staet 1에서 빠진건 : ['031', '032', '033', '041', '042', '043', '044']

### >> 로그인을 안했으니.. 로그인횟수/금액 ntile이 3분위나  4분위로 바뀌지는 않음

In [84]:
State_T1_convert['s1']= State_T1_convert['w1_kbp로그인여부']+State_T1_convert['w1_kbp로그인횟수_ntile']+State_T1_convert['w1_kbp금액_ntile']

In [85]:
State_T1_convert['s1'].unique()

array(['011', '111', '012', '112', '013', '113', '014', '114', '021',
       '121', '022', '122', '023', '123', '124', '131', '132', '133',
       '134', '141', '142', '143', '144', '024', '034'], dtype=object)

In [87]:
print(len(State_T1_convert['s1'].unique()))

25


In [88]:
c1=['0','1']
c2=['1','2','3','4']
c3=['1','2','3','4']
c_test =[a+b+c for a in c1 for b in c2 for c in c3]

In [89]:
print(len(c_test),c_test)

32 ['011', '012', '013', '014', '021', '022', '023', '024', '031', '032', '033', '034', '041', '042', '043', '044', '111', '112', '113', '114', '121', '122', '123', '124', '131', '132', '133', '134', '141', '142', '143', '144']


In [90]:
expt= [col for col in c_test if col not in State_T1_convert['s1'].unique()]

In [91]:
expt

['031', '032', '033', '041', '042', '043', '044']

- state 합치기 

In [97]:
mart_gr1_copy1.head()

,w0_kbp로그인횟수_ntile,w0_kbp금액_ntile,w0_kbp로그인여부,w1_kbp로그인횟수_ntile,w1_kbp금액_ntile,w1_kbp로그인여부,연령그룹,리워드구간대_1,리워드구간대_2,리워드구간대_3,리워드구간대_4,tst
0,1,1,1,1,1,0,0,6513.0,5630.0,2575.0,1676.0,2
1,1,1,1,1,1,0,1,16013.0,14430.0,5910.0,4018.0,2
2,1,1,1,1,1,0,2,8359.0,7400.0,3185.0,2101.0,2
3,1,1,1,1,1,1,0,1307.0,1349.0,682.0,432.0,2
4,1,1,1,1,1,1,1,2623.0,2556.0,1301.0,951.0,2


In [98]:
mart_gr1_copy2 = pd.concat([mart_gr1_copy1['연령그룹'],State_T0_convert['s0'],State_T1_convert['s1'],mart_gr1_copy1[['리워드구간대_1','리워드구간대_2','리워드구간대_3','리워드구간대_4']]],axis=1)


In [99]:
print(mart_gr1_copy1.shape, mart_gr1_copy2.shape)

(1117, 12) (1117, 7)


In [102]:
mart_gr1_copy2['전체'] =mart_gr1_copy2['리워드구간대_1']+mart_gr1_copy2['리워드구간대_2']+mart_gr1_copy2['리워드구간대_3']+mart_gr1_copy2['리워드구간대_4']
mart_gr1_copy2.head()

,연령그룹,s0,s1,리워드구간대_1,리워드구간대_2,리워드구간대_3,리워드구간대_4,전체
0,0,111,011,6513.0,5630.0,2575.0,1676.0,16394.0
1,1,111,011,16013.0,14430.0,5910.0,4018.0,40371.0
2,2,111,011,8359.0,7400.0,3185.0,2101.0,21045.0
3,0,111,111,1307.0,1349.0,682.0,432.0,3770.0
4,1,111,111,2623.0,2556.0,1301.0,951.0,7431.0


In [106]:
mart_gr1_copy2.groupby(['s0','s1']).sum()

연령그룹  리워드구간대_1  리워드구간대_2  리워드구간대_3  리워드구간대_4        전체
s0  s1                                                         
111 011     3   30885.0   27460.0   11670.0    7795.0   77810.0
    012     3   21381.0   18685.0    8075.0    5368.0   53509.0
    013     3   10552.0    9238.0    4056.0    2688.0   26534.0
    014     1       5.0       4.0       1.0       0.0      10.0
    021     3     447.0     396.0     174.0     104.0    1121.0
...       ...       ...       ...       ...       ...       ...
144 134     3    4902.0   10091.0   11236.0   13061.0   39290.0
    141     3     313.0    1975.0    3452.0    4936.0   10676.0
    142     3     157.0    1002.0    1758.0    2386.0    5303.0
    143     3    1584.0    8391.0   15240.0   20754.0   45969.0
    144     3   13386.0   45355.0   69687.0   90353.0  218781.0

[378 rows x 6 columns]

In [107]:
print(16*25)

400


### S >> S'로 변하는것도 22개가 빈다 

## 만약 State space에 연령대까지 더하면 ? 

In [110]:
mart_gr1_copy2['s0_age']= mart_gr1_copy2['s0'] + mart_gr1_copy2['연령그룹'].astype('str')
mart_gr1_copy2['s1_age']= mart_gr1_copy2['s1'] + mart_gr1_copy2['연령그룹'].astype('str')
mart_gr1_copy2.head()

,연령그룹,s0,s1,리워드구간대_1,리워드구간대_2,리워드구간대_3,리워드구간대_4,전체,s0_age,s1_age
0,0,111,011,6513.0,5630.0,2575.0,1676.0,16394.0,1110,0110
1,1,111,011,16013.0,14430.0,5910.0,4018.0,40371.0,1111,0111
2,2,111,011,8359.0,7400.0,3185.0,2101.0,21045.0,1112,0112
3,0,111,111,1307.0,1349.0,682.0,432.0,3770.0,1110,1110
4,1,111,111,2623.0,2556.0,1301.0,951.0,7431.0,1111,1111


In [112]:
print(len(mart_gr1_copy2['s0_age'].unique()),len(mart_gr1_copy2['s1_age'].unique()))

48 75


In [114]:
print(2*4*4*3)

96


####  >> s1은 전체 다 있고 s2는 21개 부족

In [115]:
mart_gr1_copy2.groupby(['s0_age','s1_age']).sum()

연령그룹  리워드구간대_1  리워드구간대_2  리워드구간대_3  리워드구간대_4       전체
s0_age s1_age                                                       
1110   0110       0    6513.0    5630.0    2575.0    1676.0  16394.0
       0120       0    4764.0    4285.0    1863.0    1194.0  12106.0
       0130       0    2402.0    2147.0     955.0     607.0   6111.0
       0140       0       3.0       0.0       0.0       0.0      3.0
       0210       0      77.0      68.0      29.0      15.0    189.0
...             ...       ...       ...       ...       ...      ...
1442   1342       2     682.0    1346.0    1649.0    1964.0   5641.0
       1412       2      61.0     426.0     785.0    1098.0   2370.0
       1422       2      38.0     207.0     401.0     536.0   1182.0
       1432       2     204.0    1400.0    2585.0    3621.0   7810.0
       1442       2    1780.0    6989.0   11921.0   16092.0  36782.0

[1117 rows x 6 columns]

In [116]:
print(48*75)

3600


#### >> 부족한 게 더 많아짐 (3600-1117)

In [49]:
tst = mart_gr.copy()

In [50]:
tst.head()

,리워드구간대,w0_kbp건수_ntile,w0_kbp금액_ntile,w0_kbp로그인횟수_ntile,w1_kbp건수_ntile,w1_kbp금액_ntile,w1_kbp로그인횟수_ntile,연령그룹,w1_kbp로그인여부_0,w1_kbp로그인여부_1
0,1,1,1,1,1,1,1,20대이하,2106.0,524.0
1,1,1,1,1,1,1,1,3040,5126.0,1198.0
2,1,1,1,1,1,1,1,50대이상,2611.0,592.0
3,1,1,1,1,1,1,2,20대이하,77.0,537.0
4,1,1,1,1,1,1,2,3040,227.0,1338.0


In [ ]:
tst_logiin_1 = tst [['w0_kbp로그인횟수_ntile''리워드구간대','w1_kbp로그인횟수_ntile','w1_kbp로그인여부']]

In [52]:
tst['w0_kbp로그인횟수_ntile'].value_counts()

3    5014
4    4945
2    4720
1    4680
Name: w0_kbp로그인횟수_ntile, dtype: int64

In [56]:
tg_rate = mart_pt1.groupby(['w1_kbp로그인여부','리워드구간대']).agg({'w1_kbp로그인여부':'count'}).unstack('w1_kbp로그인여부')

In [58]:
tg_rate

w1_kbp로그인여부        
w1_kbp로그인여부           0       1
리워드구간대                         
1                255719  299113
2                234904  582982
3                108019  683566
4                 76275  827449

In [60]:
tg_rate.columns= ['_'.join(col) for col in tg_rate.columns.values]
tg_rate

,w1_kbp로그인여부_0,w1_kbp로그인여부_1
리워드구간대,,
1,255719,299113
2,234904,582982
3,108019,683566
4,76275,827449


In [61]:
tg_rate['전체']=tg_rate['w1_kbp로그인여부_0']+tg_rate['w1_kbp로그인여부_1']
tg_rate

,w1_kbp로그인여부_0,w1_kbp로그인여부_1,전체
리워드구간대,,,
1,255719,299113,554832
2,234904,582982,817886
3,108019,683566,791585
4,76275,827449,903724


In [63]:
tg_rate['tgr_rate']=tg_rate['w1_kbp로그인여부_1']/tg_rate['전체']*100
tg_rate

,w1_kbp로그인여부_0,w1_kbp로그인여부_1,전체,tgr_rate
리워드구간대,,,,
1,255719,299113,554832,53.910553
2,234904,582982,817886,71.279127
3,108019,683566,791585,86.354087
4,76275,827449,903724,91.559923


In [54]:
tst.groupby(['w0_kbp로그인횟수_ntile','리워드구간대']).agg({'w0_kbp로그인횟수_ntile':'count'}).unstack('리워드구간대')

w0_kbp로그인횟수_ntile                  
리워드구간대                            1     2     3     4
w0_kbp로그인횟수_ntile                                    
1                              1200  1219  1152  1109
2                              1141  1192  1192  1195
3                              1175  1265  1291  1283
4                              1155  1260  1265  1265

- 필요 변수만 

In [65]:
tst.head()

,리워드구간대,w0_kbp건수_ntile,w0_kbp금액_ntile,w0_kbp로그인횟수_ntile,w1_kbp건수_ntile,w1_kbp금액_ntile,w1_kbp로그인횟수_ntile,연령그룹,w1_kbp로그인여부_0,w1_kbp로그인여부_1
0,1,1,1,1,1,1,1,20대이하,2106.0,524.0
1,1,1,1,1,1,1,1,3040,5126.0,1198.0
2,1,1,1,1,1,1,1,50대이상,2611.0,592.0
3,1,1,1,1,1,1,2,20대이하,77.0,537.0
4,1,1,1,1,1,1,2,3040,227.0,1338.0


In [68]:
tst['전체'] = tst['w1_kbp로그인여부_0']+tst['w1_kbp로그인여부_1']
tst.head()

,리워드구간대,w0_kbp건수_ntile,w0_kbp금액_ntile,w0_kbp로그인횟수_ntile,w1_kbp건수_ntile,w1_kbp금액_ntile,w1_kbp로그인횟수_ntile,연령그룹,w1_kbp로그인여부_0,w1_kbp로그인여부_1,전체
0,1,1,1,1,1,1,1,20대이하,2106.0,524.0,2630.0
1,1,1,1,1,1,1,1,3040,5126.0,1198.0,6324.0
2,1,1,1,1,1,1,1,50대이상,2611.0,592.0,3203.0
3,1,1,1,1,1,1,2,20대이하,77.0,537.0,614.0
4,1,1,1,1,1,1,2,3040,227.0,1338.0,1565.0


In [75]:
tst.pivot_table(index=['w0_kbp로그인횟수_ntile','리워드구간대'],columns='w1_kbp로그인횟수_ntile',values='전체',aggfunc='sum',margins=True)

w1_kbp로그인횟수_ntile                1         2         3         4        All
w0_kbp로그인횟수_ntile 리워드구간대                                                   
1                 1       201346.0   67646.0   11536.0    4210.0   284738.0
                  2       182072.0   67990.0   13732.0    4822.0   268616.0
                  3        80947.0   33859.0    7924.0    2557.0   125287.0
                  4        55444.0   24764.0    6152.0    2014.0    88374.0
2                 1        63167.0   71966.0   19457.0    8371.0   162961.0
                  2        63555.0  101219.0   45793.0   17992.0   228559.0
                  3        32260.0   83178.0   52464.0   17809.0   185711.0
                  4        24283.0   85147.0   61424.0   18924.0   189778.0
3                 1        12702.0   22759.0   14972.0    9492.0    59925.0
                  2        14378.0   43043.0   69379.0   38993.0   165793.0
                  3         8555.0   49641.0  119149.0   60229.0   237574.0
                  4         7007.0   59233.0  158982.0   78489.0   303711.0
4                 1         5741.0    9395.0   10197.0   21875.0    47208.0
                  2         6975.0   15662.0   37099.0   95182.0   154918.0
                  3         4515.0   15014.0   59484.0  164000.0   243013.0
                  4         4068.0   16493.0   79259.0  222041.0   321861.0
All                       767015.0  767009.0  767003.0  767000.0  3068027.0

In [77]:
tst_tranP= pd.DataFrame(tst.pivot_table(index=['w0_kbp로그인횟수_ntile','리워드구간대'],columns='w1_kbp로그인횟수_ntile',values='전체',aggfunc='sum',margins=True))
tst_tranP

w1_kbp로그인횟수_ntile                1         2         3         4        All
w0_kbp로그인횟수_ntile 리워드구간대                                                   
1                 1       201346.0   67646.0   11536.0    4210.0   284738.0
                  2       182072.0   67990.0   13732.0    4822.0   268616.0
                  3        80947.0   33859.0    7924.0    2557.0   125287.0
                  4        55444.0   24764.0    6152.0    2014.0    88374.0
2                 1        63167.0   71966.0   19457.0    8371.0   162961.0
                  2        63555.0  101219.0   45793.0   17992.0   228559.0
                  3        32260.0   83178.0   52464.0   17809.0   185711.0
                  4        24283.0   85147.0   61424.0   18924.0   189778.0
3                 1        12702.0   22759.0   14972.0    9492.0    59925.0
                  2        14378.0   43043.0   69379.0   38993.0   165793.0
                  3         8555.0   49641.0  119149.0   60229.0   237574.0
                  4         7007.0   59233.0  158982.0   78489.0   303711.0
4                 1         5741.0    9395.0   10197.0   21875.0    47208.0
                  2         6975.0   15662.0   37099.0   95182.0   154918.0
                  3         4515.0   15014.0   59484.0  164000.0   243013.0
                  4         4068.0   16493.0   79259.0  222041.0   321861.0
All                       767015.0  767009.0  767003.0  767000.0  3068027.0

In [83]:
tst_tranP.columns

Index([1, 2, 3, 4, 'All'], dtype='object', name='w1_kbp로그인횟수_ntile')

In [86]:
tst_tranP['1_P']= tst_tranP[1]/tst_tranP['All'] * 100 
tst_tranP['2_P']= tst_tranP[2]/tst_tranP['All'] * 100
tst_tranP['3_P']= tst_tranP[3]/tst_tranP['All'] * 100
tst_tranP['4_P']= tst_tranP[4]/tst_tranP['All'] * 100

In [87]:
tst_tranP

w1_kbp로그인횟수_ntile                1         2         3         4        All  \
w0_kbp로그인횟수_ntile 리워드구간대                                                      
1                 1       201346.0   67646.0   11536.0    4210.0   284738.0   
                  2       182072.0   67990.0   13732.0    4822.0   268616.0   
                  3        80947.0   33859.0    7924.0    2557.0   125287.0   
                  4        55444.0   24764.0    6152.0    2014.0    88374.0   
2                 1        63167.0   71966.0   19457.0    8371.0   162961.0   
                  2        63555.0  101219.0   45793.0   17992.0   228559.0   
                  3        32260.0   83178.0   52464.0   17809.0   185711.0   
                  4        24283.0   85147.0   61424.0   18924.0   189778.0   
3                 1        12702.0   22759.0   14972.0    9492.0    59925.0   
                  2        14378.0   43043.0   69379.0   38993.0   165793.0   
                  3         8555.0   49641.0  119149.0   60229.0   237574.0   
                  4         7007.0   59233.0  158982.0   78489.0   303711.0   
4                 1         5741.0    9395.0   10197.0   21875.0    47208.0   
                  2         6975.0   15662.0   37099.0   95182.0   154918.0   
                  3         4515.0   15014.0   59484.0  164000.0   243013.0   
                  4         4068.0   16493.0   79259.0  222041.0   321861.0   
All                       767015.0  767009.0  767003.0  767000.0  3068027.0   

w1_kbp로그인횟수_ntile               1_P        2_P        3_P        4_P  
w0_kbp로그인횟수_ntile 리워드구간대                                              
1                 1       70.712725  23.757279   4.051444   1.478552  
                  2       67.781517  25.311225   5.112130   1.795128  
                  3       64.609257  27.025150   6.324679   2.040914  
                  4       62.737909  28.021816   6.961323   2.278951  
2                 1       38.762035  44.161486  11.939667   5.136812  
                  2       27.806824  44.285721  20.035527   7.871928  
                  3       17.371077  44.788946  28.250346   9.589631  
                  4       12.795477  44.866634  32.366238   9.971651  
3                 1       21.196496  37.979141  24.984564  15.839800  
                  2        8.672260  25.961892  41.846761  23.519087  
                  3        3.600983  20.894963  50.152374  25.351680  
                  4        2.307127  19.503080  52.346474  25.843318  
4                 1       12.161074  19.901288  21.600153  46.337485  
                  2        4.502382  10.109865  23.947508  61.440246  
                  3        1.857925   6.178270  24.477703  67.486102  
                  4        1.263900   5.124262  24.625226  68.986612  
All                       25.000269  25.000073  24.999878  24.999780

In [102]:
tst_tranP.to_pickle(os.path.join(dat_path,'tst_tranP.pkl'))

### 보상 식 구하기 

In [90]:
tst_login = tst.groupby(['w0_kbp로그인횟수_ntile','리워드구간대']).agg({'전체':'sum','w1_kbp로그인여부_1':'sum'}).reset_index()
tst_login

,w0_kbp로그인횟수_ntile,리워드구간대,전체,w1_kbp로그인여부_1
0,1,1,284738.0,115149.0
1,1,2,268616.0,120164.0
2,1,3,125287.0,61347.0
3,1,4,88374.0,45475.0
4,2,1,162961.0,92825.0
5,2,2,228559.0,160939.0
6,2,3,185711.0,153972.0
7,2,4,189778.0,167455.0
8,3,1,59925.0,48096.0
9,3,2,165793.0,152261.0


In [91]:
tst_login['tg_rate'] = tst_login['w1_kbp로그인여부_1'] / tst_login['전체'] * 100
tst_login

,w0_kbp로그인횟수_ntile,리워드구간대,전체,w1_kbp로그인여부_1,tg_rate
0,1,1,284738.0,115149.0,40.440335
1,1,2,268616.0,120164.0,44.734491
2,1,3,125287.0,61347.0,48.965176
3,1,4,88374.0,45475.0,51.457442
4,2,1,162961.0,92825.0,56.961482
5,2,2,228559.0,160939.0,70.414641
6,2,3,185711.0,153972.0,82.909467
7,2,4,189778.0,167455.0,88.237309
8,3,1,59925.0,48096.0,80.260325
9,3,2,165793.0,152261.0,91.838015


In [98]:
tst_login['cost'] = tst_login['리워드구간대'].apply(lambda x : 20 if x == 1 else 57.5 if x==2 else 82.5 if x==3 else 95)

In [99]:
tst_login

,w0_kbp로그인횟수_ntile,리워드구간대,전체,w1_kbp로그인여부_1,tg_rate,cost
0,1,1,284738.0,115149.0,40.440335,20.0
1,1,2,268616.0,120164.0,44.734491,57.5
2,1,3,125287.0,61347.0,48.965176,82.5
3,1,4,88374.0,45475.0,51.457442,95.0
4,2,1,162961.0,92825.0,56.961482,20.0
5,2,2,228559.0,160939.0,70.414641,57.5
6,2,3,185711.0,153972.0,82.909467,82.5
7,2,4,189778.0,167455.0,88.237309,95.0
8,3,1,59925.0,48096.0,80.260325,20.0
9,3,2,165793.0,152261.0,91.838015,57.5


In [100]:
tst_login['reward_scen1'] = 100 - tst_login['cost']
tst_login['reward_scen2'] = 200 - tst_login['cost']
tst_login['reward_scen3'] = 300 - tst_login['cost']
tst_login['reward_scen4'] = 400 - tst_login['cost']
tst_login['reward_scen5'] = 500 - tst_login['cost']
tst_login

,w0_kbp로그인횟수_ntile,리워드구간대,전체,w1_kbp로그인여부_1,tg_rate,cost,reward_scen1,reward_scen2,reward_scen3,reward_scen4,reward_scen5
0,1,1,284738.0,115149.0,40.440335,20.0,80.0,180.0,280.0,380.0,480.0
1,1,2,268616.0,120164.0,44.734491,57.5,42.5,142.5,242.5,342.5,442.5
2,1,3,125287.0,61347.0,48.965176,82.5,17.5,117.5,217.5,317.5,417.5
3,1,4,88374.0,45475.0,51.457442,95.0,5.0,105.0,205.0,305.0,405.0
4,2,1,162961.0,92825.0,56.961482,20.0,80.0,180.0,280.0,380.0,480.0
5,2,2,228559.0,160939.0,70.414641,57.5,42.5,142.5,242.5,342.5,442.5
6,2,3,185711.0,153972.0,82.909467,82.5,17.5,117.5,217.5,317.5,417.5
7,2,4,189778.0,167455.0,88.237309,95.0,5.0,105.0,205.0,305.0,405.0
8,3,1,59925.0,48096.0,80.260325,20.0,80.0,180.0,280.0,380.0,480.0
9,3,2,165793.0,152261.0,91.838015,57.5,42.5,142.5,242.5,342.5,442.5


In [101]:
tst_login.to_pickle(os.path.join(dat_path,'tst_login.pkl'))

### transitional probability 

In [ ]:
tst.groupby(['w0_kbp로그인횟수_ntile','리워드구간대']).agg({'w0_kbp로그인횟수_ntile':'count'}).unstack('리워드구간대')